In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize
from sklearn.metrics import accuracy_score

In [2]:
def compute_euclidean(vec1, vec2):
    return np.linalg.norm(vec1 - vec2)

def compute_cosine(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_product = np.linalg.norm(vec1) * np.linalg.norm(vec2) + 1e-10
    return 1 - (dot_product / norm_product)

def compute_jaccard(vec1, vec2):
    intersection = np.sum(np.minimum(vec1, vec2))
    union = np.sum(np.maximum(vec1, vec2)) + 1e-10
    return 1 - (intersection / union)

In [ ]:
def custom_kmeans(X, num_clusters, distance_fn, max_iterations=100):
    n_samples, _ = X.shape
    centroids = X[np.random.choice(n_samples, num_clusters, replace=False)]
    
    for _ in range(max_iterations):
        groups = [[] for _ in range(num_clusters)]
        
        for idx, sample in enumerate(X):
            dists = [distance_fn(sample, center) for center in centroids]
            closest = np.argmin(dists)
            groups[closest].append(idx)
        
        new_centroids = np.zeros_like(centroids)
        for i, group in enumerate(groups):
            if group:
                new_centroids[i] = np.mean(X[group], axis=0)
            else:
                new_centroids[i] = X[np.random.choice(n_samples)]
        
        if np.allclose(centroids, new_centroids, atol=1e-6):
            break
        centroids = new_centroids

    total_sse = sum(
        distance_fn(X[i], centroids[cluster_idx]) ** 2
        for cluster_idx, group in enumerate(groups)
        for i in group
    )

    return centroids, groups, total_sse

In [4]:
features = pd.read_csv('.\kmeans_data\data.csv').values
true_labels = pd.read_csv('.\kmeans_data\label.csv').values.ravel()

In [5]:
features_normalized = normalize(features, axis=0)

num_classes = len(np.unique(true_labels))

_, _, sse_euc = custom_kmeans(features, num_classes, compute_euclidean)
_, _, sse_cos = custom_kmeans(features_normalized, num_classes, compute_cosine)
_, _, sse_jac = custom_kmeans(features_normalized, num_classes, compute_jaccard)

# Q1

In [7]:
print(f"SSE (Euclidean): {sse_euc}")
print(f"SSE (Cosine): {sse_cos}")
print(f"SSE (Jaccard): {sse_jac}")

lowest_sse = min((sse_euc, "Euclidean"), (sse_cos, "Cosine"), (sse_jac, "Jaccard"))
print(f"Lowest SSE achieved with: {lowest_sse[1]} (SSE = {lowest_sse[0]:.4f})")

SSE (Euclidean): 25370624296.0
SSE (Cosine): 1411.0988120027505
SSE (Jaccard): 4411.539120555272
Lowest SSE achieved with: Cosine (SSE = 1411.0988)


In [8]:
def assign_labels_by_majority(cluster_groups, actual_labels):
    predicted = np.zeros(len(actual_labels), dtype=int)
    for idx, group in enumerate(cluster_groups):
        if group:
            group_labels = actual_labels[group]
            majority = np.argmax(np.bincount(group_labels))
            for i in group:
                predicted[i] = majority
    return predicted

# Q2

In [9]:
centroids_euc, groups_euc, _ = custom_kmeans(features, num_classes, compute_euclidean)
centroids_cos, groups_cos, _ = custom_kmeans(features_normalized, num_classes, compute_cosine)
centroids_jac, groups_jac, _ = custom_kmeans(features_normalized, num_classes, compute_jaccard)

preds_euc = assign_labels_by_majority(groups_euc, true_labels)
preds_cos = assign_labels_by_majority(groups_cos, true_labels)
preds_jac = assign_labels_by_majority(groups_jac, true_labels)

acc_euc = accuracy_score(true_labels, preds_euc)
acc_cos = accuracy_score(true_labels, preds_cos)
acc_jac = accuracy_score(true_labels, preds_jac)

print(f"Accuracy (Euclidean): {acc_euc:.4f}")
print(f"Accuracy (Cosine): {acc_cos:.4f}")
print(f"Accuracy (Jaccard): {acc_jac:.4f}")

best_accuracy = max((acc_euc, "Euclidean"), (acc_cos, "Cosine"), (acc_jac, "Jaccard"))
print(f"Best method by accuracy: {best_accuracy[1]} ({best_accuracy[0]:.4f})")

Accuracy (Euclidean): 0.5918
Accuracy (Cosine): 0.5172
Accuracy (Jaccard): 0.5846
Best method by accuracy: Euclidean (0.5918)


In [28]:
def run_kmeans(data, k, dist_fn, max_iterations=500, tolerance=1e-6):
    num_samples, num_features = data.shape
    centers = data[np.random.choice(num_samples, k, replace=False)]

    previous_sse = float('inf')
    for iteration in range(max_iterations):
        grouped_points = [[] for _ in range(k)]
        for i, sample in enumerate(data):
            dists = [dist_fn(sample, center) for center in centers]
            assigned_cluster = np.argmin(dists)
            grouped_points[assigned_cluster].append(i)

        updated_centers = np.zeros_like(centers)
        for idx, group in enumerate(grouped_points):
            if group:
                updated_centers[idx] = np.mean(data[group], axis=0)
            else:
                updated_centers[idx] = data[np.random.choice(num_samples)]

        current_sse = sum(dist_fn(data[i], centers[cluster]) ** 2 
                          for cluster, group in enumerate(grouped_points) for i in group)

        # Stopping condition
        if np.allclose(centers, updated_centers, atol=tolerance):
            print(f"Stopped: Centroids stable at iteration {iteration + 1}")
            break
        

        centers = updated_centers
        previous_sse = current_sse

    return centers, grouped_points, current_sse


In [29]:
import time

# Q3

In [30]:
start = time.time()
centroids_euc, clusters_euc, sse_euc = run_kmeans(features, num_classes, compute_euclidean)
duration_euc = time.time() - start

start = time.time()
centroids_cos, clusters_cos, sse_cos = run_kmeans(features_normalized, num_classes, compute_cosine)
duration_cos = time.time() - start

start = time.time()
centroids_jac, clusters_jac, sse_jac = run_kmeans(features_normalized, num_classes, compute_jaccard)
duration_jac = time.time() - start

# Output runtime details
print(f"Time for Euclidean K-means: {duration_euc:.4f} seconds")
print(f"Time for Cosine K-means: {duration_cos:.4f} seconds")
print(f"Time for Jaccard K-means: {duration_jac:.4f} seconds")


Stopped: Centroids stable at iteration 76
Stopped: Centroids stable at iteration 31
Stopped: Centroids stable at iteration 68
Time for Euclidean K-means: 43.1256 seconds
Time for Cosine K-means: 29.6622 seconds
Time for Jaccard K-means: 74.7442 seconds


In [25]:
def adaptive_kmeans(data, k, dist_fn, stop_criterion, max_iterations=100, tolerance=1e-6):
    num_samples, num_features = data.shape
    centroids = data[np.random.choice(num_samples, k, replace=False)]

    last_sse = float('inf')
    for i in range(max_iterations):
        clusters = [[] for _ in range(k)]
        for j, pt in enumerate(data):
            dists = [dist_fn(pt, c) for c in centroids]
            chosen = np.argmin(dists)
            clusters[chosen].append(j)

        new_centroids = np.zeros_like(centroids)
        for idx, cluster in enumerate(clusters):
            new_centroids[idx] = np.mean(data[cluster], axis=0) if cluster else data[np.random.choice(num_samples)]

        current_sse = sum(dist_fn(data[i], centroids[cl])**2 for cl, group in enumerate(clusters) for i in group)

        if stop_criterion == 'centroid_change' and np.allclose(centroids, new_centroids, atol=tolerance):
            print(f"Centroid stability met at iteration {i+1}")
            break
        elif stop_criterion == 'sse_increase' and current_sse > last_sse:
            print(f"SSE increased at iteration {i+1}")
            break
        elif stop_criterion == 'max_iters' and i == max_iterations - 1:
            print(f"Reached iteration limit ({max_iterations})")
            break

        centroids = new_centroids
        last_sse = current_sse

    return current_sse

In [26]:
def evaluate_sse_comparisons(data, k, max_iters=100):
    criteria = ['centroid_change', 'sse_increase', 'max_iters']
    metrics = [('Euclidean', compute_euclidean), 
               ('Cosine', compute_cosine), 
               ('Jaccard', compute_jaccard)]
    
    all_sse = {}
    for name, metric_fn in metrics:
        all_sse[name] = {}
        for rule in criteria:
            result_sse = adaptive_kmeans(data, k, metric_fn, rule, max_iterations=max_iters)
            all_sse[name][rule] = result_sse
            print(f"SSE using {name} with {rule}: {result_sse:.4f}")
    return all_sse


# Q4

In [27]:
sse_summary = evaluate_sse_comparisons(features, num_classes, max_iters=100)

# Display the summarized results
print("\nOverall SSE Comparison by Metric and Stop Rule:")
for metric, results in sse_summary.items():
    print(f"\n{metric} Metric:")
    for condition, score in results.items():
        print(f"  {condition}: SSE = {score:.4f}")

Centroid stability met at iteration 52
SSE using Euclidean with centroid_change: 25511923380.0000
SSE using Euclidean with sse_increase: 25407648207.0000
Reached iteration limit (100)
SSE using Euclidean with max_iters: 25321222561.0000
Centroid stability met at iteration 57
SSE using Cosine with centroid_change: 682.1154
SSE increased at iteration 35
SSE using Cosine with sse_increase: 686.7421
Reached iteration limit (100)
SSE using Cosine with max_iters: 684.1737
Centroid stability met at iteration 44
SSE using Jaccard with centroid_change: 3682.2352
SSE increased at iteration 2
SSE using Jaccard with sse_increase: 4056.3272
Reached iteration limit (100)
SSE using Jaccard with max_iters: 3652.7958

Overall SSE Comparison by Metric and Stop Rule:

Euclidean Metric:
  centroid_change: SSE = 25511923380.0000
  sse_increase: SSE = 25407648207.0000
  max_iters: SSE = 25321222561.0000

Cosine Metric:
  centroid_change: SSE = 682.1154
  sse_increase: SSE = 686.7421
  max_iters: SSE = 684.1